In [1]:
import json
import numpy as np
import jieba
import pandas
import pickle

In [2]:
dir=r"/home/liuchang/data/"

In [3]:
raw=[]
with open(dir+"car_comments_structure.json",'r',encoding='utf8') as f:
    for line in f.readlines():
        raw.append(json.loads(line))

In [4]:
raw[0]

{'question_title': '《用小5的感受》',
 'question_forum': '骏派A50',
 'question_url': 'https://k.autohome.com.cn/detail/view_01ckzspjcr68rk6e1k6csg0000.html?st=5&piap=0%7C4404%7C0%7C0%7C2%7C0%7C0%7C0%7C0%7C0%7C1',
 '最满意': '最 满意 的 就 是 小 5 的 颜值 了 吧 ， 外观 大气 ， 三 档 以上 动力 不错 ， 空间 在 此 价位 车 里 算是 不错 的 了 ， 后备箱 够 大 。',
 '最不满意': '最 不 满意 的 就 是 内饰 有些 简陋 ， 中控台 的 塑料感 太 重 了 。 开 着 空调 怠速 不 稳 ， 一二 挡 动力 较 弱 ， 起步 肉 。',
 '空间': '空间 比较 宽敞 ， 尤其 后排 中间 是 平 的 ， 这个 比较 满意 。 乘坐 空间 ： 我 身高 1 7 0 ， 开 着 头部 距离 腿部 距离 都 比较 够用 ， 唯一 就是 座椅 有 点 硬 开 久 了 会 感觉 到 累 ， 手 舒 的 也 没有 座椅 高低 调节 。',
 '动力': '可能 是 新 车 吧 ， 还 没 磨合 出来 ， 现在 开 了 4 0 0 来 公里 ， 感觉 起步 会 有 点 肉 ， 但 能 接受 吧 ， 之后 多 磨合 下 应该 会 好些 。',
 '操控': '没 开 过 啥 好 车 ， 感觉 还是 可以 。 方向盘 很 轻 ， 有 电子 助力 ， 转 起来 也 方便 ， 单 手 就 可以 操作 。',
 '油耗': '目前 行驶 了 4 0 0 来 公里 ， 平实 都 是 上下班 ， 车 多 红绿灯 多 ， 油耗 超出 了 我 想 的 ， 就 看 首 保 过 后 了 吧 ， 应该 能 降 不 少 。',
 '舒适性': '减震 个人 感觉 偏 硬 ， 长 时间 坐 着 会 很 累 ， 不过 这 车 的 空调 挺 给力 的 ， 现在 进 车 里 几 分钟 温度 就 能 降 下去 ， 还是 挺 满意 的 。',
 '外观': '外观 是 最 满意 的 地方 了 …… 外观 设计 给 满分 ， 车 漆 稍微 薄 了 点 。',
 '内饰': '整体 

In [5]:
labels=[]
with open(dir+"/labels_dict.txt",'r',encoding='utf8') as f:
    labels=f.read().splitlines()

In [8]:
df=pandas.DataFrame([raw[i].values() for i in range(len(raw))],
                   columns=raw[0].keys())

In [9]:
df=df.astype(str)

In [10]:
df[:1]

,question_title,question_forum,question_url,最满意,最不满意,空间,动力,操控,油耗,舒适性,外观,内饰,性价比,为什么选择这款车
0,《用小5的感受》,骏派A50,https://k.autohome.com.cn/detail/view_01ckzspj...,最 满意 的 就 是 小 5 的 颜值 了 吧 ， 外观 大气 ， 三 档 以上 动力 不错...,最 不 满意 的 就 是 内饰 有些 简陋 ， 中控台 的 塑料感 太 重 了 。 开 着 ...,空间 比较 宽敞 ， 尤其 后排 中间 是 平 的 ， 这个 比较 满意 。 乘坐 空间 ：...,可能 是 新 车 吧 ， 还 没 磨合 出来 ， 现在 开 了 4 0 0 来 公里 ， 感...,没 开 过 啥 好 车 ， 感觉 还是 可以 。 方向盘 很 轻 ， 有 电子 助力 ， 转...,目前 行驶 了 4 0 0 来 公里 ， 平实 都 是 上下班 ， 车 多 红绿灯 多 ， ...,减震 个人 感觉 偏 硬 ， 长 时间 坐 着 会 很 累 ， 不过 这 车 的 空调 挺 ...,外观 是 最 满意 的 地方 了 …… 外观 设计 给 满分 ， 车 漆 稍微 薄 了 点 。,整体 风格 还是 挺 符合 我 的 口味 的 ， 简洁 大方 ， 特别 是 8 寸 大 屏幕...,价格 比较 亲民 所以 就 选择 了 小 5 。,之前 看 过 吉利 远景 ， 吉利 帝豪 ， 长 安逸动 dt 后来 因为 朋友 第一 眼 ...


In [11]:
df.to_csv(dir+"mask/df.csv",encoding='utf8')

In [12]:
comments=df[labels]

In [ ]:
comments.to_csv(dir+"mask/comments_str.csv")

In [13]:
comments

,外观,性价比,最满意,油耗,空间,最不满意,操控,动力,舒适性,内饰,为什么选择这款车
0,外观 是 最 满意 的 地方 了 …… 外观 设计 给 满分 ， 车 漆 稍微 薄 了 点 。,价格 比较 亲民 所以 就 选择 了 小 5 。,最 满意 的 就 是 小 5 的 颜值 了 吧 ， 外观 大气 ， 三 档 以上 动力 不错...,目前 行驶 了 4 0 0 来 公里 ， 平实 都 是 上下班 ， 车 多 红绿灯 多 ， ...,空间 比较 宽敞 ， 尤其 后排 中间 是 平 的 ， 这个 比较 满意 。 乘坐 空间 ：...,最 不 满意 的 就 是 内饰 有些 简陋 ， 中控台 的 塑料感 太 重 了 。 开 着 ...,没 开 过 啥 好 车 ， 感觉 还是 可以 。 方向盘 很 轻 ， 有 电子 助力 ， 转...,可能 是 新 车 吧 ， 还 没 磨合 出来 ， 现在 开 了 4 0 0 来 公里 ， 感...,减震 个人 感觉 偏 硬 ， 长 时间 坐 着 会 很 累 ， 不过 这 车 的 空调 挺 ...,整体 风格 还是 挺 符合 我 的 口味 的 ， 简洁 大方 ， 特别 是 8 寸 大 屏幕...,之前 看 过 吉利 远景 ， 吉利 帝豪 ， 长 安逸动 dt 后来 因为 朋友 第一 眼 ...
1,看中 就是 外观 ， 比 大众 朗逸 还要 漂亮 ，,还 不错 吧 比 我 预算 的 价格 差不多,空间 大 一 家 人 足够 ， 底盘 高 ， 比 吉利 帝豪 还要 高 那么 多 ，,刚刚 买 来 油耗 还 不 是 很 理想 ， 但是 我 在 磨合期 ， 油耗 也 在 慢慢 ...,空间 一 家 人 足足 有余 了 ， 后排 坐 三 个 成年人 刚刚 好 ， 不怎么 拥挤 ...,让 人 不 满意 就 是 急 加速 发动机 噪音 大 ， 超过 二千 转 就 好 多 了 ，...,驾驶感 还是 不错 的 ， 我 喜欢 电子 转向 ， 很 轻盈 ， 指 哪 打 那 ， 还 ...,车 毕竟 2 0 0 0 多 斤 1 . 5 排量 ， 5 0 不 开 空调 动力 还 不错...,减震 比较 硬 ， 胎 噪 有 点 大 ， 隔音 效果 不 是 很 好 ， 跑 到 6 0 ...,还 可以 塑料 材质 比较 硬,第一 眼 就 开 上 它 的 外观 ， 和 花瓣 轮毂 ， 空间 大 ， 价格 合理 配置 ...
2,乔治罗亚 设计 的 ， 虽然 没 听说 过 这个 人 ， 不过 外观 确实 设计 到位 了 ...,性价比 就 不用 说 了 ， 买 车 不 能 只 看 指导价 ， 除了 少数 车 ， 一般 ...,家人 乘坐 觉得 满意 ， 安全性 ， 性价比 ， 外观 ， 油耗 这些 。 （ 我家人 一...,在 山城 和 数量 众多 的 红绿灯 里 开 出 纯 市区 7 . 8 的 油耗 我 还 能...,副驾 后 排装 了 背篼 ， 坐 进去 离 前排 后背 还有 1 拳 2 指 的 距离 ， ...,发动机 隔音 有待 改进 ， 风噪胎 噪 都 控制 得 很 好 ， 为什么 就 发动机 噪音...,操控 得心应手 ， 速度 提高 方向盘 逐渐 沉重 ， 指向 精准 ， 无 虚 位 ， 转 ...,1 . 5 排量 自 吸 发动机 ， 你 想 它 很 强劲 是 不 可能 ， 但是 家用 足...,父母 评价 说 坐 着 还 不错 ， 没有 空间 很 紧张 压抑 的 感觉 。 我 父亲 因...,内饰 布局 满意 ， 不过 能 不 能 别 全 是 塑料 ？ ？ ？ 加 个 避光垫 和 各...,从 觉得 需要 有 辆 车 开始 ， 进 了 之 家 学习 了 几 个 月 ， 辗转 在 多...
3,外观 高大 上,这 价格 这 配置 ， 性价比 很 高,外观 和 油耗 ， 外观 大气 上档次 ， 油耗 目前 还 没 出 磨合期 ， 油耗 高 点,目前 行驶 4 0 0 多 公里 ， 油耗 不 低 ， 希望 后期 会 降 下来 ，,后排 坐 三 个 人 无 压力 ， 我 最 多 拉 过 七 个 人 ， 当然 有 两 个 小孩 ，,车皮 有 点 薄 ， 内饰 真心 不好 ， 全 塑料 ， 好在 我 对 内饰 要求 不 高,还 行,还 行 ， 开 空调 后 加速 费劲,一般 般,内饰 一般 ， 硬 塑料 太 多 ， 也许 是 出于 成本 考虑,价格 低 ， 配置 高 ， 外观 来 看 怎么 也 不 会 像 是 五万 多 块 钱 的 车...
4,看 过 的 人 都 说 好 ， 连 途观 的 车主 都 觉得 途观 不如 大科 漂亮 。 林...,还 说 什么 呢 ， 不 就 冲 着 性价 比 去 的 么 ， 车 也 买 了 ， 只 期待...,外观 漂亮 应 首当其冲 ， 饱满 而 不 觉 臃肿 ， 富实 而 不 显 笨重 ， 沉稳 ...,目前 仅 跑 了 5 0 0 来 公里 ， 磨合期 未 过 ， 油耗 不 是 很 准 ， 约...,本人 全家 个子 均 不 高 ， 因此 空间 绰绰有余 ， 后 排翘二郎腿 ， 脚尖 不 碰...,与众车友 同感 ， 塑料 件 大量 采用 ， 内饰 简洁 掩盖 不 了 简陋 ， 尤 以 车...,很 舒服 ， 前面 说 过 了 。 自己 开 过 才 知道,以前 我 第一 辆 车 是 1 . 6 自 吸 发动机 ， 我 都 能 开 出 推背感 ， ...,稳 而 硬 ， 过 减速 带 感觉 明显 ， 不如 冠道 和 urv 舒服 ， 其实 也 可...,大科 是 跟 哪家 塑料厂 有 战略 合作 关系 么 ？ 用 了 这么 多 塑料 ， 搪塑 ...,挑 车 数 月 ， 花落大科 ， 理由 有 三 ： 其一 ， 只 图 实惠 不 慕 虚名 。...
5,外观 属于 耐 看型 ， 个人 比较 喜欢 ， 不 花哨 ， 比较 喜欢 ！ 但 个人 觉得...,性价 比较 好 ， 满意 ！ 大众 的 配置 ， 斯柯达 的 实用 ！ 唯一 不好 就是 品...,空间 大 、 实用性 强 、 斯柯达 也 是 有 历史 的 品牌 、 油耗 低 、 底盘 扎...,算 过 实际 油耗 ， 期间 高速 、 山路 都 跑 过 ， 没有 超过 9 个 ， 8 个...,1 . 7 m 身高 ， 乘坐 宽敞 ， 后排 空间 更 棒 ， 空间 优势 非常 出色 ！ ！,前车轮 遇到 坑 ， 有时 会 发出 撞击 异响 ， 这种 异响 会 给 人 带来 不 安全感 。,指向 清晰 ， 盘子 稍 重 ， 但是 本人 喜欢 这种 感觉 。 唯一 觉得 不好 的 应...,动力 不错 ， 走过 高速 、 开 个 山路 、 上 过 陡坡 、 车上 坐 满 人 也 会...,舒适度 一般 ， 没什么 毛病 ， 也 没 发现 优点 。,一般 我 觉得 还 行 。 实用 就 行 ， 本人 不 喜欢 那种 太 花里胡哨 的 。,性价比 ， 实用 ， 适合 自己 ， 爱 我 所 爱 。 选 他 首先 是 实用性 ， 然后...
6,一直 很 喜欢 大熊 的 外观 ， 中 大型 suv 有棱有角 非常 的 硬派 风格 ， 但...,同样 对比 过 rx 5 柯珞克 标志 5 0 0 8 等 车型 ， 只能 说 各 有 优点...,动力 源源不断 ， 后排 座椅 可 调整 靠 背 角度 ， 车 内 空间 非常 大 ， ac...,油耗 新 车 的 关系 有点 大 ， 但是 第一 箱 油 加 满 了 6 0 升 左右 跑 ...,本人 身材 1 6 8 ， 坐 进 车 内 空间 非常 巨大 ， 后排 也 很 舒适 可以 ...,整车 内饰 塑料件 太 多 特别 是 门把手 关门 开门 都 是 咔咔咔 的 ， 低速 变速...,因为 是 suv 车 身高 的 关系 所以 在 高速 紧急 变道 会 有 点 飘逸感 ， 跑...,因为 车 大和 重 的 关系 想 省油 的 慢慢 踩 会 觉得 很 弱 ， 但是 加大 油门...,车 提 回来 研究 了 半天 原来 前面 两 个 不 是 雾灯 ， 只是 转弯 辅助 照明 ...,内饰 有点 欠缺 的 就 是 门板 拉手 ， 开门 关门 碰到 就是 塑料 硬碰硬 的 声音...,本来 就是 轿车 空间 太 小 置换 大 suv 的 初衷
7,流线 好 。 c 型 尾灯 钻石 切割 。 不错 。 就是 尾部 下方 黑色 塑料 的 防刮...,总体 的 性能 和 配置 加上 这个 价钱 已经 是 物有所值 了 。 四驱合资 suv ，...,动力 充沛 ， 运动档 更 是 狂暴 ， eco 模式 比 想象 中 省油 。 空间 大 ，...,目前 行驶 5 0 0 0 km ， 油耗 9 . 7 。 比 之前 的 捷达 肯定 是 高...,空间 真的 很 大 ， 还有 储物 空间 很 贴心 很 实用 。 放 行李 ： 帮 朋友 搬...,门把手 异 响 ， 史诗 级 的 吐槽 。 车 内 灯光 为 卤素 黄光 自己 改 了 le...,转向 精准 。 路感 反馈 明确 ， 能 实时 了解 路况 。 后 悬架 比 前 悬架 偏 ...,eco 有些 慢 ， d 档 中 规 中 矩 ， s 档 心跳 加速 eco 就 是 省油 ...,前排 座椅 两侧 翼 如果 能 再 往 中间 靠近 5 厘米 就 更 好 了 。 我 已经 ...,门把手 异 响 ， 木纹 不 是 木 ， 是 塑料 。 车门 不 是 皮 是 塑料 。 瑭塑..

In [17]:
join=lambda s: ''.join(s.split(' '))
comments_joined=comments.applymap(join)

In [18]:
comments_list=comments_joined.applymap(lambda s: list(jieba.cut(s)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.751 seconds.
Prefix dict has been built succesfully.


In [ ]:
comments_list.to_csv(dir+'mask/comments_list.csv')

In [22]:
with open(dir+'mask/comments_list.csv','r',encoding='utf8') as f:
    comments_list=pandas.read_csv(f,index_col=0)

In [6]:
with open(dir+'mask/comments_joined.pickle','rb') as f:
    comments_joined=pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/liuchang/data/mask/comments_joined.pickle'

In [ ]:
comments_list['外观'][0]

0                 是 最 满意 的 地方 了 … … 外观设计 给 满分 ， 车漆 稍微 薄 了 点
1                                     就是 外观 ， 比 大众 朗逸 还要 漂亮
2          罗亚 设计 的 ， 虽然 没听说过 这个 人 ， 不过 外观 确实 设计 到位 了 ， 前...
3                                                        高大
4          的 人 都 说好 ， 连途观 的 车主 都 觉得 途观 不如 大科 漂亮 。 林荫道 上 ...
5          属于 耐看 型 ， 个人 比较 喜欢 ， 不 花哨 ， 比较 喜欢 ！ 但 个人 觉得 车...
6          很 喜欢 大熊 的 外观 ， 中 大型 suv 有棱有角 非常 的 硬派 风格 ， 但是 ...
7          好 。 c 型 尾灯 钻石 切割 。 不错 。 就是 尾部 下方 黑色 塑料 的 防 刮板...
8         设计 年轻 ， 皮厚 、 led 大灯 ， 转向 辅助 灯 、 线条 绕车 一圈 的 计设 ...
9                        敦实 ， 这个 大家 都 有 自己 的 审美观 ， 不用 多 说 了
10                                            漂亮 啊 ， 是 我 喜欢
11                                                       不错
12         霸气 不失 运动 风 ， 像 我 快 30 的 人 了 感觉 途观 l 太 老气 ， 50...
13                                                 霸气 ， 前脸 
14         非常 喜欢 ， 但是 外观 造型 仁者见仁 智者 简直 。 车身 造型 ： 城市 suv ...
15                                                       比较
16                             的 审美观 不 一

In [17]:
comments_list=comments_list.applymap(lambda s: s[2:-2])

In [15]:
comments_list=comments_list.applymap(lambda s: s.replace("', '"," "))

In [21]:
comments_str=comments_list

In [8]:
m=lambda x: x if len(x)>=5 and len(x)<=80 else np.nan
comments_list_5_80=comments_list.applymap(m)

In [9]:
comments_list_5_80.to_csv(dir+'mask/comments_list_5_80.csv')

In [16]:
comments_str_5_80=comments_list_5_80.applymap(lambda s:' '.join(s) if isinstance(s,list) else s)

In [17]:
import re

In [51]:
comments_segmented_5_40=comments_str_5_40.applymap(lambda s:re.split('[，。]',re.sub('[!?;！？；]|… …|…','，',s)) if isinstance(s,str) else s)

In [56]:
comments_segmented_5_40.to_csv(dir+'mask/comments_segmented_5_40.csv')

In [22]:
cl=comments_separated_list_5_40.values.flatten()

cl=[l for l in cl if isinstance(l,list)]

cl=[[s for s in l if s.strip()!='']for l in cl  ]
cl=[l for l in cl if len(l)>2]

In [23]:
def mask_subsentence(lists, mask_index=None,mode='random'):
    if not mask_index:
        print("No mask_index,check mode.")
        mask_index = []
        if mode == 'random':
            for l in lists:
                mask_index.append(np.random.randint(1, len(l)))
        elif mode == 'median':
            mask_index = [len(l) // 2 for l in lists]
        else:
            raise ValueError('mode should be "random" or "median"')
    labels = [lists[i][mask_index[i]] for i in range(len(lists))]
    corpus = ['，'.join(lists[i][:mask_index[i]] + [' <mask> '] + lists[i][mask_index[i] + 1:]) for i in range(len(lists))]

    return corpus, labels

In [24]:
corpus,labels=mask_subsentence(cl)

No mask_index,check mode.


In [ ]:
with open(dir+'mask/corpus.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(corpus))

In [ ]:
with open(dir+'mask/labels.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(labels))

In [27]:
length=len(labels)

In [ ]:
with open(dir+'mask/train_corpus.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(corpus[:int(0.8*length)]))
with open(dir+'mask/eval_corpus.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(corpus[int(0.8*length):int(0.9*length)]))
with open(dir+'mask/test_corpus.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(corpus[int(0.9*length):]))

In [ ]:
with open(dir+'mask/train_labels.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(labels[:int(0.8*length)]))
with open(dir+'mask/eval_labels.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(labels[int(0.8*length):int(0.9*length)]))
with open(dir+'mask/test_labels.txt','w',encoding='utf8') as f:
    f.writelines('\n'.join(labels[int(0.9*length):]))

In [47]:
def compare_result(preds,labels,corpus,output='compare.txt'):
    with open(preds,'r') as f:
        pl=f.read().splitlines()
        if pl[-1]=='':
            pl.pop()
    with open(labels,'r') as f:
        ll=f.read().splitlines()
        if ll[-1]=='':
            ll.pop()
    with open(corpus,'r') as f:
        cl=f.read().splitlines()
        if cl[-1]=='':
            cl.pop()
    if len(pl)!=len(ll) or len(ll)!=len(cl):
        raise ValueError("predictions,labels and corpus should have same length.")
    result=[]
    for p,l,c in zip(pl,ll,cl):
        result.append(c.replace('mask',l +" | "+p))
    with open(output,'w',encoding='utf8') as f:
        f.writelines('\n'.join(result))

In [53]:
dir2=r"/data/share/liuchang/car_comment/mask/"


In [54]:
preds=dir2+'pred_100000.txt'
labels=dir2+'test_labels.txt'
corpus=dir2+'test_corpus.txt'

In [50]:
compare_result(preds,labels,corpus,dir2+'compare.txt')